# 차원 축소


## 1.환경준비

### (1) 라이브러리 로딩

In [1]:
# 기본 라이브러리 가져오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

from sklearn.datasets import load_breast_cancer, load_digits, load_iris, make_swiss_roll
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA

### (2) 샘플 데이터 생성하기


In [8]:
# 럭비공 형태의 샘플 데이터 생성 함수
def generate_rugby_data(n_points=1000, a=1, b=1.5, c=2):
    phi = np.random.uniform(0, np.pi, n_points)
    theta = np.random.uniform(0, 2*np.pi, n_points)
    x = a * np.sin(phi) * np.cos(theta)
    y = b * np.sin(phi) * np.sin(theta)
    z = c * np.cos(phi)
    X = np.column_stack((x, y, z))
    return X

rugby = generate_rugby_data()

# 스위스롤 데이터
swiss_roll, _ = make_swiss_roll(n_samples=1000, noise=0.2)

In [9]:
# 3차원 스캐터 함수 생성
def my_3d_Scatter(X) :
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2],
                           mode='markers', marker=dict(size=2, color='blue'),
                           name='Original Data'))

    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                      scene=dict(xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis'))

    fig.show()

In [16]:
import nbformat
print(nbformat.__version__)

5.10.4


## 2.PCA 개념이해

### (1) 럭비공 형태의 데이터 차원 축소

* 원본 데이터 둘러보기

In [20]:
my_3d_Scatter(rugby)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

* 차원 축소

In [ ]:
# PCA를 이용하여 2개의 주성분으로 차원 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(rugby)

# PCA 축소 데이터 조회
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.grid()
plt.show()


### (2) 스위스롤 형태의 데이터 차원 축소

* 원본 데이터 둘러보기

In [ ]:
my_3d_Scatter(swiss_roll)

* 차원 축소

In [ ]:
# PCA를 이용하여 2개의 주성분으로 차원 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(swiss_roll)

# PCA 축소 데이터 조회
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.grid()
plt.show()

## 3.PCA 사용해보기

### (1) 데이터 준비

* 데이터 로딩

In [ ]:
iris = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/iris.csv")
target = 'Species'
x = iris.drop(target, axis = 1)
y = iris.loc[:, target]

In [ ]:
x.head()

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x2 = scaler.fit_transform(x)

# (옵션)데이터프레임 변환
x2 = pd.DataFrame(x2, columns= x.columns)

### (2) 주성분 분석

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature 수
x2.shape[1]

* 주 성분 분석 수행

In [ ]:
# 주성분 수 2개
n = 2
pca = PCA(n_components = n)

# 만들고, 적용하기(결과는 넘파이 어레이)
x2_pc = pca.fit_transform(x2)

In [ ]:
# 2개의 주성분
x2_pc[:5]

In [ ]:
# (옵션) 데이터프레임으로 변환
x2_pc = pd.DataFrame(x2_pc, columns = ['PC1', 'PC2'])
x2_pc.head()

* 기존 데이터에 차원 축소된 데이터를 붙여 봅시다.

In [ ]:
pd.concat([iris, x2_pc], axis = 1).head()

* 두개의 주성분 시각화

In [ ]:
sns.scatterplot(x = 'PC1', y = 'PC2', data = x2_pc, hue = y)
plt.grid()
plt.show()

## 4.고차원 데이터 차원축소

### (1) 데이터 준비

#### 1) 데이터 로딩

In [ ]:
# breast_cancer 데이터 로딩
cancer=load_breast_cancer()
x = cancer.data
y = cancer.target

x = pd.DataFrame(x, columns=cancer.feature_names)

x.shape

In [ ]:
x.head()

In [ ]:
x.info()

In [ ]:
x.describe().T

#### 2) 스케일링
* 거리계산 기반 차원축소이므로 스케일링 필요

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

# (옵션)데이터프레임 변환
x = pd.DataFrame(x, columns=cancer.feature_names)

#### 3) 데이터 분할

* train, validation 분할

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (2) 주성분 만들기

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature 수
x_train.shape[1]

* 주 성분 분석 수행

In [ ]:
# 주성분을 몇개로 할지 결정(최대값 : 전체 feature 수)
n = x_train.shape[1]

# 주성분 분석 선언
pca = PCA(n_components=n)

# 만들고, 적용하기
x_train_pc = pca.fit_transform(x_train)
x_val_pc = pca.transform(x_val)

* 편리하게 사용하기 위해 데이터프레임으로 변환

In [ ]:
# 칼럼이름 생성
column_names = [ 'PC'+str(i+1) for i in range(n) ]
column_names

In [ ]:
# 데이터프레임으로 변환하기
x_train_pc = pd.DataFrame(x_train_pc, columns = column_names)
x_val_pc = pd.DataFrame(x_val_pc, columns = column_names)
x_train_pc

<img src='https://raw.githubusercontent.com/jangrae/img/master/practice_01.png' width=120 align="left"/>

* 문1) 다음의 조건으로 주성분을 추출해 봅시다.
    * 주성분 1개로 선언하고, x_train을 이용해서 주성분 추출
    * 주성분 2개로 선언하고, x_train을 이용해서 주성분 추출
    * 주성분 3개로 선언하고, x_train을 이용해서 주성분 추출

In [ ]:
# 주성분 1개짜리


In [ ]:
# 주성분 2개짜리


In [ ]:
# 주성분 3개짜리


* 문2) 각 주성분 결과에서 상위 3개 행씩 조회하여 비교해 봅시다.

### (3) 주성분 누적 분산 그래프
* 그래프를 보고 적절한 주성분의 개수를 지정(elbow method!)
* x축 : PC 수
* y축 : 전체 분산크기 - 누적 분산크기

In [ ]:
plt.plot(range(1,n+1), pca.explained_variance_ratio_, marker = '.')
plt.xlabel('No. of PC')
plt.grid()
plt.show()

주성분 개수 몇개면 충분할까요?

### (4) 시각화
* 주 성분 중 상위 2개를 뽑아 시각화 해 봅시다.

In [ ]:
sns.scatterplot(x = 'PC1', y = 'PC2', data = x_train_pc, hue = y_train)
plt.grid()
plt.show()

## 5.지도학습으로 연계하기

### (1) 원본데이터로 모델 생성하기
* knn 알고리즘으로 분류 모델링을 수행합니다.
* k : 기본값으로 지정

* 학습

In [ ]:
model0 = KNeighborsClassifier()
model0.fit(x_train, y_train)

* 예측 및 평가

In [ ]:
# 원본데이터 모델의 성능
pred0 = model0.predict(x_val)

print(confusion_matrix(y_val, pred0))
print(accuracy_score(y_val, pred0))
print(classification_report(y_val, pred0))

### (2) 실습
* 다음의 조건으로 모델을 만들고 성능을 확인해 봅시다.
    * 알고리즘 : KNN

#### 1) 주성분 상위 1개로 모델 만들기

#### 2) 주성분 2개로 모델링

## 6.[추가]t-SNE

### (1) 학습(차원축소)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# 2차원으로 축소하기
tsne = TSNE(n_components = 2, random_state=20)
x_tsne = tsne.fit_transform(x)

# 사용의 편리함을 위해 DataFrame으로 변환
x_tsne = pd.DataFrame(x_tsne, columns = ['T1','T2'])

In [ ]:
x_tsne.shape

### (2) 시각화

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x = 'T1', y = 'T2', data = x_tsne, hue = y)
plt.grid()

### (3) 실습

#### 1) 데이터 준비

* 샘플데이터 로딩

In [ ]:
digits = load_digits()
x = digits.data
y = digits.target

y = pd.Categorical(y)

In [ ]:
x.shape

* 둘러보기

In [ ]:
print(x[0].reshape(8,8))

In [ ]:
# f, axes = plt.subplots(5, 2, sharey=True, figsize=(16,6))
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(x[i,:].reshape([8,8]), cmap='gray');

* 스케일링

In [ ]:
# 최대, 최소값
np.min(x), np.max(x)

In [ ]:
# 최대값으로 나누면 Min Max 스케일링이 됩니다.
x = x / 16

#### 2) PCA
* 주성분 2개로 차원을 축소하고
* 시각화 합니다.

In [ ]:
# 차원 축소
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x)

# 데이터프레임으로 변환(옵션)
x_pca = pd.DataFrame(x_pca, columns = ['PC1', 'PC2'])

In [ ]:
# 시각화
plt.figure(figsize=(8, 8))
sns.scatterplot(x = 'PC1', y = 'PC2', data = x_pca, hue = y)
plt.grid()
plt.show()

#### 3) tSNE
* 2차원으로 축소하고
* 시각화 합니다.

In [ ]:
tsne = TSNE(n_components = 2, random_state=20)
x_tsne = tsne.fit_transform(x)

# 데이터프레임으로 변환(옵션)
x_tsne = pd.DataFrame(x_tsne, columns = ['T1', 'T2'])

In [ ]:
# 시각화
plt.figure(figsize=(8, 8))
sns.scatterplot(x = 'T1', y = 'T2', data = x_tsne, hue = y)
plt.grid()
plt.show()